# Testing Merging MapBox Traffic data into OSM Graph

In the previous step before this we were testing using Osmium to pull nodes and edges from an OSM pbf file and creating a networkX graph. In this testing notebook we will be importing the graph we created and merge MapBox traffic data to it.

In [1]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTNets")
import GOSTnets as gn

# pip install osmium
import osmium, logging
import shapely.wkb as wkblib

from shapely.geometry import LineString, Point

import time

In [4]:
# Load MapBox traffic data
in_folder = "./osm"

In [11]:
traffic_csv_1233300 = os.path.join(in_folder, "1233300-Asia-Colombo.csv")
traffic_1233300 = pd.read_csv(traffic_csv_1233300, header = None)
traffic_1233300

,0,1,2,3,4,5,6,7,8,9,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1148494884,4177608798,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
1,1148495298,4137314867,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57
2,1242700523,6537570627,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
3,1242730766,3377418986,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
4,1243299175,3805435746,40,40,40,40,40,40,40,40,...,40,40,40,40,40,40,40,40,40,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24246,849231590,2920467254,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
24247,901010011,2058634444,47,47,47,47,47,47,47,47,...,47,47,47,47,47,47,47,47,47,47
24248,901010134,3377392482,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
24249,904233206,1242795139,24,24,24,24,24,24,24,24,...,24,24,24,24,24,24,24,24,24,24


In [14]:
type(traffic_1233300)

pandas.core.frame.DataFrame

In [5]:
traffic_csv_1233302 = os.path.join(in_folder, "1233302-Asia-Colombo.csv")
traffic_1233302 = pd.read_csv(traffic_csv_1233302, header = None)
traffic_1233302

,0,1,2,3,4,5,6,7,8,9,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,102738212,3279663552,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49
1,1093698450,1093697790,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
2,1093698671,1093697934,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
3,1093798472,1093799200,39,39,39,39,39,39,39,39,...,39,39,39,39,39,39,39,39,39,39
4,1093798791,1093799519,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467858,937250922,2334155507,41,41,41,41,41,41,41,41,...,41,41,41,41,41,41,41,41,41,41
467859,98553797,6409372992,50,50,50,50,50,50,50,50,...,50,50,50,50,50,50,50,50,50,50
467860,98554190,3814750237,47,47,47,47,47,47,47,47,...,47,47,47,47,47,47,47,47,47,47
467861,98557759,2155638061,36,36,36,36,36,36,36,36,...,36,36,36,36,36,36,36,36,36,36


In [12]:
traffic_csv_1233303 = os.path.join(in_folder, "1233303-Asia-Colombo.csv")
traffic_1233303 = pd.read_csv(traffic_csv_1233303, header = None)
traffic_1233303

,0,1,2,3,4,5,6,7,8,9,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1147027495,3318178916,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
1,1147029008,3333484939,39,39,39,39,39,39,39,39,...,39,39,39,39,39,39,39,39,39,39
2,1147030124,3318163467,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
3,1675585654,1675585568,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
4,1675667479,5787866342,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9921,6495299720,6495344002,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
9922,6495398243,6495398223,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
9923,6899782766,3317976522,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
9924,7082570748,7082570747,66,66,66,66,66,66,66,66,...,66,66,66,66,66,66,66,66,66,66


In [16]:
type(traffic_1233300)

pandas.core.frame.DataFrame

In [17]:
type(traffic_1233302)

pandas.core.frame.DataFrame

In [29]:
traffic_csv_merged = traffic_1233300.append(traffic_1233302, ignore_index = True, verify_integrity = True).append(traffic_1233303, ignore_index = True, verify_integrity = True)

In [30]:
traffic_csv_merged

,0,1,2,3,4,5,6,7,8,9,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1148494884,4177608798,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
1,1148495298,4137314867,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57
2,1242700523,6537570627,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
3,1242730766,3377418986,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
4,1243299175,3805435746,40,40,40,40,40,40,40,40,...,40,40,40,40,40,40,40,40,40,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502035,6495299720,6495344002,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
502036,6495398243,6495398223,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
502037,6899782766,3317976522,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
502038,7082570748,7082570747,66,66,66,66,66,66,66,66,...,66,66,66,66,66,66,66,66,66,66


In [31]:
def get_speeds(x):
    ''' Return Min, Max, and Mean speed '''
    x_vals = x[2:]
    return([min(x_vals), max(x_vals), np.mean(x_vals)]) #, np.argmax(x_vals)
    
traffic_vals = traffic_csv_merged.apply(lambda x: get_speeds(x), axis = 1, result_type = "expand")
traffic_vals.columns = ['min_speed','max_speed','mean_speed']

In [32]:
traffic_simplified = traffic_csv_merged.loc[:,[0,1]]
traffic_simplified.columns = ['FROM_NODE', "TO_NODE"]
traffic_simplified = traffic_simplified.join(traffic_vals)

In [33]:
# read Osmium produced graph
G = nx.read_gpickle("./osmium_graph.pickle")

## convert graph into geodataframes and merge travel times

In [34]:
start_time = time.time()

edges_gdf = gn.edge_gdf_from_graph(G)
nodes_gdf = gn.node_gdf_from_graph(G)

In [37]:
edges_gdf

,stnode,endnode,Wkt,infra_type,length,osm_id,geometry
0,50366374,3177819227,"LINESTRING (80.1097108 7.1555452, 80.109937400...",trunk,0.000229,485635126,"LINESTRING (80.10994 7.15558, 80.10971 7.15555)"
1,50366374,3505017714,"LINESTRING (80.10993740000001 7.1555769, 80.11...",trunk,0.000438,485635126,"LINESTRING (80.10994 7.15558, 80.11037 7.15554)"
2,50366374,6543300362,"LINESTRING (80.1100047 7.1559307, 80.109937400...",residential,0.000360,696757131,"LINESTRING (80.10994 7.15558, 80.11000 7.15593)"
3,50366379,4432629275,"LINESTRING (80.1025528 7.1517464, 80.1025812 7...",trunk,0.000034,485635126,"LINESTRING (80.10258 7.15177, 80.10255 7.15175)"
4,50366379,3783146983,"LINESTRING (80.1025812 7.1517657, 80.103235299...",trunk,0.000795,485635126,"LINESTRING (80.10258 7.15177, 80.10324 7.15222)"
...,...,...,...,...,...,...,...
954369,7369784287,2026484619,"LINESTRING (79.9221434 7.0273885, 79.9221629 7...",residential,0.000202,192089639,"LINESTRING (79.92214 7.02739, 79.92216 7.02719)"
954370,7369784287,7369784286,"LINESTRING (79.9221434 7.0273885, 79.9218163 7...",residential,0.000329,788258759,"LINESTRING (79.92214 7.02739, 79.92182 7.02736)"
954371,7369784288,1370719823,"LINESTRING (79.923225 7.0271382, 79.9231475 7....",residential,0.000268,192089642,"LINESTRING (79.92323 7.02714, 79.92315 7.02739)"
954372,7369784289,1370719823,"LINESTRING (79.9231475 7.0273948, 79.9231151 7...",residential,0.000110,192089642,"LINESTRING (79.92312 7.02750, 79.92315 7.02739)"


In [35]:
attributed_edges = edges_gdf.merge(traffic_simplified, left_on=['stnode','endnode'], right_on=['FROM_NODE','TO_NODE'])

end_time = time.time()
print(end_time - start_time)

57.264134883880615


After this you would still need to convert to a graph from the edge and node GeoDataFrames somehow...

### Now add mapbox traffic speeds directly to the graph

In [36]:
start_time = time.time()

def add_edge_attribute(x):
    G[x['stnode']][x['endnode']][0]['mapbox_traffic_speed'] = int(x['mean_speed'])
    G[x['endnode']][x['stnode']][0]['mapbox_traffic_speed'] = int(x['mean_speed'])
    
attributed_edges.apply(lambda x: add_edge_attribute(x), axis = 1)

end_time = time.time()
print(end_time - start_time)

6.388365983963013
